In [1]:
# Copyright (c) Stef van der Struijk
# License: GNU Lesser General Public License
import sys
sys.path.append(".")
import copy
from controller import Controller
from ipywidgets import interact, interactive, interactive_output, HBox, Label, Layout
import ipywidgets as widgets

In [2]:
# restart kernel if changes made to .py file
controller = Controller(pub_ip="127.0.0.1", pub_port=5570, pub_key="command.multiplier", pub_bind=False)  # 192.168.11.12

Current libzmq version is 4.2.5
Current  pyzmq version is 17.0.0
Publisher port is specified
Creating ZeroMQ context on: tcp://127.0.0.1:5570
Connect to tcp://127.0.0.1:5570 successful
Publisher socket set-up complete
port_sub not specified, not setting-up subscriber
ZeroMQ sockets successfully set-up

init done


In [3]:
# AU sliders
au_descriptions = {
    "AU01": "Inner brow raiser",
    "AU02": "Outer brow raiser",
    "AU04": "Brow lowerer",
    "AU05": "Upper lid raiser",
    "AU06": "Cheek raiser",
    "AU07": "Lid tightener",
    "AU09": "Nose wrinkler",
    "AU10": "Upper lip raiser",
    "AU12": "Lip corner puller",
    "AU14": "Dimpler",
    "AU15": "Lip corner depressor",
    "AU17": "Chin raiser",
    "AU20": "Lip stretcher",
    "AU23": "Lip tightener",
    "AU25": "Lips part",
    "AU26": "Jaw drop",
    "AU45": "Blink"
}

style_au = {'description_width': '190px'}


# emotion sliders
emo_list = [
    "Happiness",
    "Sadness",
    "Surprise",
    "Fear",
    "Anger",
    "Disgust"
]

style_emo = {'description_width': '80px'}


# get all AUs in new dict
au_dict_emo = copy.deepcopy(au_descriptions)
# remove blink from modifiers
au_dict_emo.pop('AU45', None)
# print(au_dict_emo)

'Blink'

In [4]:
# set facial configuration in real-time

# create float sliders for every AU
dict_config = {}
for au, desc in au_descriptions.items():
    dict_config[au] = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.02, style=style_au,
                                      layout=Layout(width='520px'),
                                      description="{} ({})".format(au, desc))
    
# create float sliders for head pose
for pose in ["pose_Rx", "pose_Ry", "pose_Rz"]:
    dict_config[pose] = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.02, style=style_au,
                                      layout=Layout(width='520px'))

# send facial configuration to FACSvatar
def view_face_configuration(**kwargs):
    controller.face_configuration(kwargs)
    # return kwargs
    

    
# create float sliders for every AU
emo_dict_config = {}

# exagerate expressions
emo_dict_config['Exaggerate'] = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.05, style=style_emo,
                                      layout=Layout(width='400px'))

# per emotion a float slider
for emo in emo_list:
    emo_dict_config[emo] = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05, style=style_emo,
                                  layout=Layout(width='400px'))


# modfiy multiple AUs at the same time
def emotion_au_config(**kwargs):
#     print(kwargs)
    
    # reset all AU values
    for au, val in au_dict_emo.items():
        au_dict_emo[au] = 0.0
        
    # happiness
    au_dict_emo["AU06"] += kwargs['Happiness']
    au_dict_emo["AU12"] += kwargs['Happiness']
    
    # Sadness
    au_dict_emo["AU01"] += kwargs['Sadness']
    au_dict_emo["AU04"] += kwargs['Sadness']
    au_dict_emo["AU15"] += kwargs['Sadness']
    
    # Surprise
    au_dict_emo["AU01"] += kwargs['Surprise']
    au_dict_emo["AU02"] += kwargs['Surprise']
    au_dict_emo["AU05"] += kwargs['Surprise']
    au_dict_emo["AU26"] += kwargs['Surprise']
    
    # Fear
    au_dict_emo["AU01"] += kwargs['Fear']
    au_dict_emo["AU02"] += kwargs['Fear']
    au_dict_emo["AU04"] += kwargs['Fear']
    au_dict_emo["AU05"] += kwargs['Fear']
    au_dict_emo["AU07"] += kwargs['Fear']
    au_dict_emo["AU20"] += kwargs['Fear']
    au_dict_emo["AU26"] += kwargs['Fear']
    
    # Anger
    au_dict_emo["AU04"] += kwargs['Anger']
    au_dict_emo["AU05"] += kwargs['Anger']
    au_dict_emo["AU07"] += kwargs['Anger']
    au_dict_emo["AU23"] += kwargs['Anger']
    
    # Disgust
    au_dict_emo["AU09"] += kwargs['Disgust']
    au_dict_emo["AU15"] += kwargs['Disgust']
    au_dict_emo["AU25"] += kwargs['Disgust']
    
    
    # set AU slider values * exaggerate multiplier
    for au, val in au_dict_emo.items():
        dict_config[au].value = val * kwargs['Exaggerate']
    
    return kwargs



# display sliders
# w_face = interactive(view_face_configuration, **dict_config)
# display(w_face)

w_face = interactive(view_face_configuration, **dict_config)
w_emo_config = interactive(emotion_au_config, **emo_dict_config)

# print(w.children)
display(HBox([w_face, w_emo_config]))

In [5]:
# multiply (increase / decrease) AU values in real-time
print("AU multiplier sliders")

# create float sliders for every AU
au_dict_multi = {}
for au, desc in au_descriptions.items():
    au_dict_multi[au] = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.05, style=style_au,
                                      layout=Layout(width='520px'),
                                      description="{} ({})".format(au, desc))
    
# increase blink multiplier by default
au_dict_multi['AU45'].value = 1.5

# print(au_dict_multi['AU01'].style.keys)

# send multiplier values to FACSvatar
def view_multiplier(**kwargs):
    controller.multiplier(kwargs)
    #return kwargs
    

# create float sliders for every AU
emo_dict_multi = {}

# exagerate expressions
emo_dict_multi['Exaggerate'] = widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.05, style=style_emo,
                                      layout=Layout(width='400px')) # , description="Exaggerate"

# per emotion a float slider
for emo in emo_list:
    emo_dict_multi[emo] = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.05, style=style_emo,
                                  layout=Layout(width='400px'))
    


# modfiy multiple AUs at the same time
def emotion_au_multi(**kwargs):
#     print(kwargs)
    
    # reset all AU values
    for au, val in au_dict_emo.items():
        au_dict_emo[au] = 1.0
        
    # happiness
    au_dict_emo["AU06"] += kwargs['Happiness']
    au_dict_emo["AU12"] += kwargs['Happiness']
    
    # Sadness
    au_dict_emo["AU01"] += kwargs['Sadness']
    au_dict_emo["AU04"] += kwargs['Sadness']
    au_dict_emo["AU15"] += kwargs['Sadness']
    
    # Surprise
    au_dict_emo["AU01"] += kwargs['Surprise']
    au_dict_emo["AU02"] += kwargs['Surprise']
    au_dict_emo["AU05"] += kwargs['Surprise']
    au_dict_emo["AU26"] += kwargs['Surprise']
    
    # Fear
    au_dict_emo["AU01"] += kwargs['Fear']
    au_dict_emo["AU02"] += kwargs['Fear']
    au_dict_emo["AU04"] += kwargs['Fear']
    au_dict_emo["AU05"] += kwargs['Fear']
    au_dict_emo["AU07"] += kwargs['Fear']
    au_dict_emo["AU20"] += kwargs['Fear']
    au_dict_emo["AU26"] += kwargs['Fear']
    
    # Anger
    au_dict_emo["AU04"] += kwargs['Anger']
    au_dict_emo["AU05"] += kwargs['Anger']
    au_dict_emo["AU07"] += kwargs['Anger']
    au_dict_emo["AU23"] += kwargs['Anger']
    
    # Disgust
    au_dict_emo["AU09"] += kwargs['Disgust']
    au_dict_emo["AU15"] += kwargs['Disgust']
    au_dict_emo["AU25"] += kwargs['Disgust']
    
    
    # set AU slider values * exaggerate multiplier
    for au, val in au_dict_emo.items():
        au_dict_multi[au].value = val * kwargs['Exaggerate']
    
    return kwargs
   
    
    
w_multi = interactive(view_multiplier, **au_dict_multi)
w_emo = interactive(emotion_au_multi, **emo_dict_multi)

# print(w.children)
display(HBox([w_multi, w_emo]))

AU multiplier sliders


In [6]:
# set AU multiplier
# @interact(
#     AU01=(0.0,2.0,0.05),
#     AU02=(0.0,2.0,0.05),
#     AU04=(0.0,2.0,0.05),
#     AU05=(0.0,2.0,0.05),
#     AU06=(0.0,2.0,0.05),
#     AU07=(0.0,2.0,0.05),
#     AU09=(0.0,2.0,0.05),
#     AU10=(0.0,2.0,0.05),
#     AU12=(0.0,2.0,0.05),
#     AU14=(0.0,2.0,0.05),
#     AU15=(0.0,2.0,0.05),
#     AU17=(0.0,2.0,0.05),
#     AU20=(0.0,2.0,0.05),
#     AU23=(0.0,2.0,0.05),
#     AU25=(0.0,2.0,0.05),
#     AU26=(0.0,2.0,0.05),
#     AU45=(0.0,2.0,0.05))
# def f(**kwargs):
#     # print(type(kwargs['AU25']))
#     controller.multiplier(kwargs)
#     #return kwargs